# Extracting an area based on a shapefile

## Section 1 

In [ ]:
# Initial imports and setup
import sys
import os, re
import datacube
import datetime as dt
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.colors
import matplotlib as mpl
import cmocean # for intertidal products
import odc.geo.xr # RML
import pandas as pd
import rioxarray
import warnings
import xarray as xr

#from dea_tools.landcover import lc_colourmap, make_colorbar, plot_land_cover
from dea_tools.plotting import display_map, rgb, xr_animation # Added xr_animation RML

from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog
from datacube.utils.masking import mask_invalid_data
from datacube.utils.geometry import Geometry
from datetime import datetime, timedelta
from ipyleaflet import DrawControl, FullScreenControl, LayersControl, Map, Rectangle, WidgetControl, basemaps # Added rectangle RML
from ipywidgets import Button, Dropdown, FloatText, GridspecLayout, Layout, HBox, VBox, Output, Tab, interact, widgets
from IPython.display import HTML, display
from IPython.core.display import Video
from pyproj import Proj, transform
from shapely.geometry import Polygon
from shapely.geometry.polygon import LinearRing
from time import time
warnings.filterwarnings("ignore")

# Setup paths and Datacube instance
#import sys
#sys.path.insert(1, "/home/Tools")

dc = datacube.Datacube()
time_range = None

sys.path.append("../Working")
import display_tools
from display_tools import display_da
from landcover import lc_colourmap, make_colorbar, plot_land_cover

# Ensure openpyxl is installed
!pip install openpyxl


In [ ]:
# Additional libraries etc.

from datacube.utils.geometry import Geometry, CRS
from ipyleaflet import GeoData
from display_tools import map_geom, rgb

sys.path.append("../Tools/wdc_tools")
from wdc_datahandling import geopolygon_masking

import ipywidgets as widgets
import glob
import os
import geopandas as gpd

In [ ]:
vector_types_list = glob.glob("uploads/*")
vector_types_dict = {}
for vector_type in vector_types_list:
    if os.path.isdir(vector_type):
        vector_types_dict[os.path.basename(vector_type).replace("_"," ")] = vector_type

In [ ]:
vector_types_dict

In [ ]:
get_type = widgets.Dropdown(
    options=list(vector_types_dict.keys()),
    value=list(vector_types_dict.keys())[0],
    description="Type",
    disabled=False,
)

## Section 2 Load up a polygon for your area of interest

In [ ]:
# Open and read the shapefiles
Mungalla_path = 'uploads/Mungalla-outline.shp'
Mungalla = gpd.read_file(Mungalla_path)

In [ ]:
Starcke_path = 'uploads/Starcke-outline1.shp'
Starcke = gpd.read_file(Starcke_path)

In [ ]:
Mungalla.head(3)

In [ ]:
Starcke.head(3)

In [ ]:
# Transform shapefile boundaries into geographic data (and affect a style) 
geo_data = GeoData(geo_dataframe = Mungalla.to_crs(epsg=4326),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 
                          'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')

# map the geographic data on dynamic map
m = map_geom(geo_data)
m

In [ ]:
geom = Geometry(geom=Mungalla.iloc[0].geometry, 
                         crs=CRS("epsg:32755"))
geom

In [ ]:
# Transform shapefile boundaries into geographic data (and affect a style) 
geo_data = GeoData(geo_dataframe = Starcke.to_crs(epsg=4326),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 
                          'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')

# map the geographic data on dynamic map
m = map_geom(geo_data)
m

In [ ]:
# Transform shapefile boundaries into geographic data (and affect a style) 
geo_data = GeoData(geo_dataframe = Starcke.to_crs(epsg=4326),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 
                          'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')

# map the geographic data on dynamic map
m = map_geom(geo_data)
m

In [ ]:
geom = Geometry(geom=Starcke.iloc[0].geometry, 
                         crs=CRS("epsg:32755"))
geom

In [ ]:
query = {'geopolygon': geom
        }

In [ ]:
# Load land cover data for our polygon and time period
#lc_dataset = dc.load(product='lw_landcover_lw', **query)
#lc_dataset_masked = geopolygon_masking(lc_dataset, geopolygon=geom)

# Step 3: Load DEA Land Cover data from the datacube
try:
    lc = dc.load(
        product="ga_ls_landcover_class_cyear_2",
        output_crs="EPSG:32755",
        measurements=[
            "level3",
            "lifeform",
            "vegetation_cover",
            "water_seasonality",
            "water_state",
            "intertidal",
            "water_persistence",
            "bare_gradation",
            "full_classification",
        ],
        resolution=(-25, 25),
        **query
    )
    print("Data loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading data from the datacube: {e}")
    
lc_dataset_masked = geopolygon_masking(lc, geopolygon=geom)

In [ ]:
# Load land cover data for our polygon and time period
#lc_dataset = dc.load(product='lc', **query)
lc_masked = geopolygon_masking(lc, geopolygon=geom)

In [ ]:
lc

In [ ]:
# Section 6: Checkbox widgets for selecting datasets
level3_checkbox = widgets.Checkbox(description='Level 3', value=False)
lifeform_checkbox = widgets.Checkbox(description='Lifeform', value=False)
vegetation_cover_checkbox = widgets.Checkbox(description='Vegetation Cover', value=False)
water_state_checkbox = widgets.Checkbox(description='Water State', value=False)
water_persistence_checkbox = widgets.Checkbox(description='Water Persistence', value=False)
full_classification_checkbox = widgets.Checkbox(description='Full Classification', value=False)
item_checkbox = widgets.Checkbox(description='Intertidal Elevation', value=False)

output = widgets.Output()

def plot_selected_datasets():
    with output:
        output.clear_output()
        selected_datasets = []
        if level3_checkbox.value:
            selected_datasets.append('level3')
        if lifeform_checkbox.value:
            selected_datasets.append('lifeform')
        if vegetation_cover_checkbox.value:
            selected_datasets.append('vegetation_cover')
        if water_state_checkbox.value:
            selected_datasets.append('water_state')
        if water_persistence_checkbox.value:
            selected_datasets.append('water_persistence')            
        if full_classification_checkbox.value:
            selected_datasets.append('full_classification')
        if item_checkbox.value:
            selected_datasets.append('item')

        for dataset in selected_datasets:
            plt.figure()
            if dataset == 'level3':
                plot_land_cover(lc.level3)
            elif dataset == 'lifeform':
                plot_land_cover(lc.lifeform)
            elif dataset == 'vegetation_cover':
                plot_land_cover(lc.vegetation_cover)
            elif dataset == 'water_state':
                plot_land_cover(lc.water_state)
            elif dataset == 'water_persistence':
                plot_land_cover(lc.water_persistence)
            elif dataset == 'full_classification':
                plot_land_cover(lc.full_classification)
            elif dataset == 'item':
                plot_land_cover(item.elevation[0])
            plt.show()

plot_datasets_button = widgets.Button(description='Plot Selected Data')
plot_datasets_button.on_click(lambda b: plot_selected_datasets())

checkboxes_box = widgets.HBox([
    level3_checkbox, 
    lifeform_checkbox, 
    vegetation_cover_checkbox, 
    water_state_checkbox, 
    water_persistence_checkbox, 
    full_classification_checkbox,
    item_checkbox
])
display(checkboxes_box)
display(plot_datasets_button)
display(output)